In [2]:
# General libraries
import numpy as np
import numba as nb
from numba import jit, njit, prange
# from numba import cuda, vectorize
import pandas as pd

In [3]:
# no. of banks
N = 15
# time step
T = 3000
# sample size
size = 10000

# Bayesian Learning
L_max = 50
B_max = 5
LSM_max = 50

In [4]:
samples = pd.read_csv("homobanks_0.5.csv") 
n_sample = samples['Sample'].to_numpy()
time = samples['Time'].to_numpy()
bank = samples['Bank'].to_numpy().astype('intc')
payee = samples['Payee'].to_numpy().astype('intc')
pri = samples['Priority'].to_numpy()
amount = samples['Amount'].to_numpy()

In [5]:
# Find the index of the start of each sample (for threading)
@jit(nopython = True)
def find_ind():
    index = np.zeros(size, np.intc)
    i = 0
    for j in range(len(n_sample)):
        if i < size:
            if i == n_sample[j]:
                
                index[i] = np.intc(j)
                i += 1
        else:
            return index

index = find_ind()


In [ ]:
@jit(nopython = True, parallel = True)
def tensor_gen():    
    
    _liquidity_tensor = np.zeros((L_max + 1) * (B_max + 1) * (L_max * (N-1) + 1) * (B_max * (N - 1) + 1) * N).reshape((L_max + 1, B_max + 1, L_max * (N-1) + 1, B_max * (N - 1)+1, N))
    buffer_tensor = np.zeros((L_max + 1) * (B_max + 1) * (L_max * (N-1) + 1) * (B_max * (N - 1) + 1) * N).reshape((L_max + 1, B_max + 1, L_max * (N-1) + 1, B_max * (N - 1)+1, N))
    
    #Create the _liquidity_tensor and buffer_tensor
    for i in prange(L_max + 1):

        for k in prange(B_max + 1):
            _liquidity_tensor[i][k][0][0] = np.zeros(N)
            _liquidity_tensor[i][k][0][0][0] = i

            for j in range(L_max * (N-1)+1):
                buffer_tensor[i][k][j][0] = np.zeros(N)
                buffer_tensor[i][k][j][0][0] = k
                if j > 0:
                    _liquidity_tensor[i][k][j][0] = _liquidity_tensor[i][k][j-1][0]
                    _liquidity_tensor[i][k][j][0][(j-1) % (N - 1)  + 1] += 1

                for l in range(B_max * (N - 1) + 1): 
                    if l > 0:
                        _liquidity_tensor[i][k][j][l] = _liquidity_tensor[i][k][j][l-1]
                        buffer_tensor[i][k][j][l] = buffer_tensor[i][k][j][l-1]
                        buffer_tensor[i][k][j][l][(l - 1) % (N - 1) + 1] += 1
                        
    return _liquidity_tensor, buffer_tensor

_liquidity_tensor, buffer_tensor = tensor_gen()

In [20]:
@jit(nopython = True)
def simulation(n, _lt, b):
    # delay of priority and non-priority payments
    p_delay = np.zeros(N)
    np_delay = np.zeros(N)

    ip_delay = np.zeros(N)
    inp_delay = np.zeros(N)
    
    # counts of payments queuing
    pq_count = np.zeros(N, np.intc)
    npq_count = np.zeros(N, np.intc)
    
    ipq_count = np.zeros(N, np.intc)
    inpq_count = np.zeros(N, np.intc)
    
    # starting location of the queue
    pq_s = np.zeros(N, np.intc)
    ipq_s = np.zeros(N, np.intc)
    
    npq_s = np.zeros(N, np.intc)
    inpq_s = np.zeros(N, np.intc)
    
    # store the timing of the queue
    pq_loc = np.zeros(np.intc(N * T/3)).reshape((np.intc(N), np.intc(T/3))).astype(np.intc)
    npq_loc = np.zeros(np.intc(N * T/3)).reshape((np.intc(N), np.intc(T/3))).astype(np.intc)

    ipq_loc = np.zeros(np.intc(N * T/3)).reshape((np.intc(N), np.intc(T/3))).astype(np.intc)
    inpq_loc = np.zeros(np.intc(N * T/3)).reshape((np.intc(N), np.intc(T/3))).astype(np.intc)

    # record to track subsequent payments
    trans = np.zeros(N, np.intc)
    itrans = np.zeros(N, np.intc)
    
    t = index[n]
 
    i_lt = np.copy(_lt)

    ###########################################
    
    while n == n_sample[t]: # same sample
        # case order arrivals
        # priority order
        if pri[t] == 1:
            if amount[t] <= _lt[bank[t]] and pq_count[bank[t]] == 0:
                _lt[bank[t]] -= amount[t]
                _lt[payee[t]] += amount[t]
                trans[payee[t]] = 1
            else:
                
                pq_loc[bank[t]][pq_s[bank[t]] + pq_count[bank[t]]] = t
                pq_count[bank[t]] += 1
                                
            if 0 < i_lt[bank[t]] and ipq_count[bank[t]] == 0:
                i_lt[bank[t]] -= 1
                i_lt[payee[t]] += 1
                itrans[payee[t]] = 1
            else:
                
                ipq_loc[bank[t]][ipq_s[bank[t]] + ipq_count[bank[t]]] = t
                ipq_count[bank[t]] += 1                     

        # non-priority order  
        elif pri[t] == 0:
            if amount[t] + b[bank[t]] <= _lt[bank[t]] and (pq_count[bank[t]] == 0 and npq_count[bank[t]] == 0):
                _lt[bank[t]] -= amount[t]
                _lt[payee[t]] += amount[t]
                trans[payee[t]] = 1
            else:              
                
                npq_loc[bank[t]][npq_s[bank[t]] + npq_count[bank[t]]] = t
                npq_count[bank[t]] += 1
                        
            if b[bank[t]] < i_lt[bank[t]] and (ipq_count[bank[t]] == 0 and inpq_count[bank[t]] == 0):
                i_lt[bank[t]] -= 1
                i_lt[payee[t]] += 1
                itrans[payee[t]] = 1
            else:              
                
                inpq_loc[bank[t]][inpq_s[bank[t]] + inpq_count[bank[t]]] = t
                inpq_count[bank[t]] += 1

 
 #######################################################################################
        # case of receiving liquidity
        while np.max(trans) > 0:
            p_trans = np.argmax(trans)
            
            if pq_count[p_trans] != 0:
                _t = pq_loc[p_trans][pq_s[p_trans]]
                if amount[_t] <= _lt[p_trans]:
                    # transaction
                    _lt[p_trans] -= amount[_t]
                    _lt[payee[_t]] += amount[_t]
                    p_delay[p_trans] += (time[t] - time[_t]) * amount[_t]

                    trans[payee[_t]] = 1
                    
                    pq_s[p_trans] += 1
                    pq_count[p_trans] -= 1
                    
                else:
                    trans[p_trans] = 0
                    
            elif pq_count[p_trans] == 0 and npq_count[p_trans] != 0:
                _t = npq_loc[p_trans][npq_s[p_trans]]          
                if amount[_t] + b[p_trans] <= _lt[p_trans]:
                    _lt[p_trans] -= amount[_t]
                    _lt[payee[_t]] += amount[_t]
                    np_delay[p_trans] += (time[t] - time[_t]) * amount[_t]

                    trans[payee[_t]] = 1
                    
                    npq_s[p_trans] += 1
                    npq_count[p_trans] -= 1
                    
                else:
                    trans[p_trans] = 0
            
            else:
                trans[p_trans] = 0

            
        while np.max(itrans) > 0:
            ip_trans = np.argmax(itrans)

            if ipq_count[ip_trans] != 0:
                _t = ipq_loc[ip_trans][ipq_s[ip_trans]]
                if 0 < i_lt[ip_trans]:
                    # transaction
                    i_lt[ip_trans] -= 1
                    i_lt[payee[_t]] += 1
                    ip_delay[ip_trans] += (time[t] - time[_t])

                    itrans[payee[_t]] = 1
                    
                    ipq_s[ip_trans] += 1
                    ipq_count[ip_trans] -= 1
                    
                else: 
                    itrans[ip_trans] = 0

            
            elif ipq_count[ip_trans] == 0 and inpq_count[ip_trans] != 0:
                _t = inpq_loc[ip_trans][inpq_s[ip_trans]]          
                if b[ip_trans] < i_lt[ip_trans]:
                    
                    i_lt[ip_trans] -= 1
                    i_lt[payee[_t]] += 1
                    inp_delay[ip_trans] += (time[t] - time[_t])

                    itrans[payee[_t]] = 1
                    
                    inpq_s[ip_trans] += 1
                    inpq_count[ip_trans] -= 1
                
                else: 
                    itrans[ip_trans] = 0
                    
            else: 
                itrans[ip_trans] = 0

# ###################################################################################################
        t += 1
        if t == len(n_sample):
            break

    # Force settlement

    k = 0
    total = np.float64(0)
    # clear the payments in the queue
    for l in range(pq_count[k]):
        _t = pq_loc[k][pq_s[k] + l]
        if _lt[k] > 0:
            total += amount[_t] - _lt[k]
            _lt[k] = 0
        else:
            total += amount[_t]   
        p_delay[k] += (T - time[_t]) * amount[_t]

    for l in range(npq_count[k]):
        _t = npq_loc[k][npq_s[k] + l]
        if _lt[k] > amount[_t]:
            _lt[k] -= amount[_t]
        elif _lt[k] > 0 and _lt[k] < amount[_t]:
            total += amount[_t] - _lt[k]
            _lt[k] = 0
        elif _lt[k] == 0:
            total += amount[_t]
        np_delay[k] += (T - time[_t]) * amount[_t]

    itotal = np.float64(0)
    # clear the payments in the queue
    for l in range(ipq_count[k]):
        _t = ipq_loc[k][ipq_s[k] + l]
        itotal += 1                                  
        ip_delay[k] += (T - time[_t])

    for l in range(inpq_count[k]):
        _t = inpq_loc[k][inpq_s[k] + l]
        if i_lt[k] > 0:
            i_lt[k] -= 1

        elif i_lt[k] == 0:
            itotal += 1
        inp_delay[k] += (T - time[_t])             

    return np.array([p_delay[k], np_delay[k], total, ip_delay[k], inp_delay[k], itotal])
    

In [ ]:
# The threading function
@jit(nopython = True, parallel=True)
def Thread_sim(_liquidity, buffer):

    for n in prange(1):
        result = simulation(n, _liquidity, buffer)
        p_delay += result[0]
        np_delay += result[1]
        total += result[2]
        ip_delay += result[3]
        inp_delay += result[4]
        itotal += result[5]

    return p_delay/size, np_delay/size, total/size, ip_delay/size, inp_delay/size, itotal/size

519 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
print(bank[20], payee[20])

In [ ]:
# @jit(nopython = True, parallel = True)
# def LSM_tensor_gen():    

#     _liquidity_tensor = np.zeros((LSM_max  + 1) * (LSM_max * (N-1) + 1) * N).reshape((LSM_max + 1, LSM_max * (N-1) + 1, N))

#     #Create the _liquidity_tensor
#     for i in prange(LSM_max + 1):
#         _liquidity_tensor[i][0][0] = i
        
#         for j in range(LSM_max * (N-1)+1):
#             if j > 0:
#                 _liquidity_tensor[i][j] = _liquidity_tensor[i][j-1]
#                 _liquidity_tensor[i][j][(j-1) % (N-1)  + 1] += 1

#     return _liquidity_tensor

# LSM_liquidity_tensor = LSM_tensor_gen()